---
**Remix Note**: This ELI5 version was created with the Applied Learning AI Notebooks (ALAIN) Project on 09.14.2025.\nCreated by [Daniel Green](https://www.linkedin.com/in/danielpgreen).\n---



In [ ]:
# Environment Detection
import sys
IN_COLAB = 'google.colab' in sys.modules
print(f'Environment: {"Colab" if IN_COLAB else "Local"}')


In [ ]:
# 🔧 Environment Detection and Setup
import sys
import os

# Detect environment
IN_COLAB = 'google.colab' in sys.modules
env_label = 'Google Colab' if IN_COLAB else 'Local'
print(f'Environment: {env_label}')

# Setup environment-specific configurations
if IN_COLAB:
    print('📝 Colab-specific optimizations enabled')
    try:
        from google.colab import output
        output.enable_custom_widget_manager()
    except Exception:
        pass


## API Keys and .env Files\n\nMany providers require API keys. Do not hardcode secrets in notebooks. Use a local .env file that the notebook loads at runtime.\n\n- Why .env? Keeps secrets out of source control and tutorials.\n- Where? Place `.env.local` (preferred) or `.env` in the same folder as this notebook. `.env.local` overrides `.env`.\n- What keys? Common: `POE_API_KEY` (Poe-compatible servers), `OPENAI_API_KEY` (OpenAI-compatible), `HF_TOKEN` (Hugging Face).\n- Find your keys:\n  - Poe-compatible providers: see your provider's dashboard for an API key.\n  - Hugging Face: create a token at https://huggingface.co/settings/tokens (read scope is usually enough).\n  - Local servers: you may not need a key; set `OPENAI_BASE_URL` instead (e.g., http://localhost:1234/v1).\n\nThe next cell will: load `.env.local`/`.env`, prompt for missing keys, and optionally write `.env.local` with secure permissions so future runs just work.

In [ ]:
# 🔐 Load and manage secrets from .env\n# This cell will: (1) load .env.local/.env, (2) prompt for missing keys, (3) optionally write .env.local (0600).\n# Location: place your .env files next to this notebook (recommended) or at project root.\n# Disable writing: set SAVE_TO_ENV = False below.\nimport os, pathlib\nfrom getpass import getpass\n\n# Install python-dotenv if missing\ntry:\n    import dotenv  # type: ignore\nexcept Exception:\n    import sys, subprocess\n    if 'IN_COLAB' in globals() and IN_COLAB:\n        try:\n            import IPython\n            ip = IPython.get_ipython()\n            if ip is not None:\n                ip.run_line_magic('pip', 'install -q python-dotenv>=1.0.0')\n            else:\n                subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'python-dotenv>=1.0.0'])\n        except Exception as colab_exc:\n            print('⚠️ Colab pip fallback failed:', colab_exc)\n            raise\n    else:\n        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'python-dotenv>=1.0.0'])\n    import dotenv  # type: ignore\n\n# Prefer .env.local over .env\ncwd = pathlib.Path.cwd()\nenv_local = cwd / '.env.local'\nenv_file = cwd / '.env'\nchosen = env_local if env_local.exists() else (env_file if env_file.exists() else None)\nif chosen:\n    dotenv.load_dotenv(dotenv_path=str(chosen))\n    print(f'Loaded env from {chosen.name}')\nelse:\n    print('No .env.local or .env found; will prompt for keys.')\n\n# Keys we might use in this notebook\nkeys = ['POE_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']\nmissing = [k for k in keys if not os.environ.get(k)]\nfor k in missing:\n    val = getpass(f'Enter {k} (hidden, press Enter to skip): ')\n    if val:\n        os.environ[k] = val\n\n# Decide whether to persist to .env.local for convenience\nSAVE_TO_ENV = True  # set False to disable writing\nif SAVE_TO_ENV:\n    target = env_local\n    existing = {}\n    if target.exists():\n        try:\n            for line in target.read_text().splitlines():\n                if not line.strip() or line.strip().startswith('#') or '=' not in line:\n                    continue\n                k,v = line.split('=',1)\n                existing[k.strip()] = v.strip()\n        except Exception:\n            pass\n    for k in keys:\n        v = os.environ.get(k)\n        if v:\n            existing[k] = v\n    lines = []\n    for k,v in existing.items():\n        # Always quote; escape backslashes and double quotes for safety\n        escaped = v.replace("\\", "\\\\")\n        escaped = escaped.replace("\"", "\\"")\n        vv = f'"{escaped}"'\n        lines.append(f"{k}={vv}")\n    target.write_text('\\n'.join(lines) + '\\n')\n    try:\n        target.chmod(0o600)  # 600\n    except Exception:\n        pass\n    print(f'🔏 Wrote secrets to {target.name} (permissions 600)')\n\n# Simple recap (masked)\ndef mask(v):\n    if not v: return '∅'\n    return v[:3] + '…' + v[-2:] if len(v) > 6 else '•••'\nfor k in keys:\n    print(f'{k}:', mask(os.environ.get(k)))\n

In [ ]:
# 🌐 ALAIN Provider Setup (Poe/OpenAI-compatible)
# About keys: If you have POE_API_KEY, this cell maps it to OPENAI_API_KEY and sets OPENAI_BASE_URL to Poe.
# Otherwise, set OPENAI_API_KEY (and optionally OPENAI_BASE_URL for local/self-hosted servers).
import os
try:
    # Prefer Poe; fall back to OPENAI_API_KEY if set
    poe = os.environ.get('POE_API_KEY')
    if poe:
        os.environ.setdefault('OPENAI_BASE_URL', 'https://api.poe.com/v1')
        os.environ.setdefault('OPENAI_API_KEY', poe)
    # Prompt if no key present
    if not os.environ.get('OPENAI_API_KEY'):
        from getpass import getpass
        os.environ['OPENAI_API_KEY'] = getpass('Enter POE_API_KEY (input hidden): ')
        os.environ.setdefault('OPENAI_BASE_URL', 'https://api.poe.com/v1')
    # Ensure openai client is installed
    try:
        from openai import OpenAI  # type: ignore
    except Exception:
        import sys, subprocess
        if 'IN_COLAB' in globals() and IN_COLAB:
            try:
                import IPython
                ip = IPython.get_ipython()
                if ip is not None:
                    ip.run_line_magic('pip', 'install -q openai>=1.34.0')
                else:
                    cmd = [sys.executable, "-m", "pip", "install", '-q', 'openai>=1.34.0']
                    try:
                        subprocess.check_call(cmd)
                    except Exception as exc:
                        if IN_COLAB:
                            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
                            if packages:
                                try:
                                    import IPython
                                    ip = IPython.get_ipython()
                                    if ip is not None:
                                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                                    else:
                                        import subprocess as _subprocess
                                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                                except Exception as colab_exc:
                                    print('⚠️ Colab pip fallback failed:', colab_exc)
                                    raise
                            else:
                                print('No packages specified for pip install; skipping fallback')
                        else:
                            raise
            except Exception as colab_exc:
                print('⚠️ Colab pip fallback failed:', colab_exc)
                raise
        else:
            cmd = [sys.executable, "-m", "pip", "install", '-q', 'openai>=1.34.0']
            try:
                subprocess.check_call(cmd)
            except Exception as exc:
                if IN_COLAB:
                    packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
                    if packages:
                        try:
                            import IPython
                            ip = IPython.get_ipython()
                            if ip is not None:
                                ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                            else:
                                import subprocess as _subprocess
                                _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                        except Exception as colab_exc:
                            print('⚠️ Colab pip fallback failed:', colab_exc)
                            raise
                    else:
                        print('No packages specified for pip install; skipping fallback')
                else:
                    raise
        from openai import OpenAI  # type: ignore
    # Create client
    from openai import OpenAI
    client = OpenAI(base_url=os.environ['OPENAI_BASE_URL'], api_key=os.environ['OPENAI_API_KEY'])
    print('✅ Provider ready:', os.environ.get('OPENAI_BASE_URL'))
except Exception as e:
    print('⚠️ Provider setup failed:', e)


In [ ]:
# 🔎 Provider Smoke Test (1-token)
import os
model = os.environ.get('ALAIN_MODEL') or 'gpt-4o-mini'
if 'client' not in globals():
    print('⚠️ Provider client not available; skipping smoke test')
else:
    try:
        resp = client.chat.completions.create(model=model, messages=[{"role":"user","content":"ping"}], max_tokens=1)
        print('✅ Smoke OK:', resp.choices[0].message.content)
    except Exception as e:
        print('⚠️ Smoke test failed:', e)


> Generated by ALAIN (Applied Learning AI Notebooks) — 2025-09-16.


# GPT-5 Prompting for Absolute Beginners: A Playground Guide

Imagine GPT-5 as an eager new friend who can fetch facts, write stories, or build tiny apps—if you ask clearly. This notebook teaches non-coders how to steer that friend with simple, repeatable “magic phrases” so you always get helpful, on-topic answers and avoid confusing tangents.


> ⏱️ Estimated time to complete: 36–60 minutes (rough).  
> 🕒 Created (UTC): 2025-09-16T03:49:46.231Z



## Learning Objectives

By the end of this tutorial, you will be able to:

1. Explain what a prompt is using toy-box analogies (no jargon).
2. Write three short prompts that tell GPT-5 exactly how eager or cautious it should be.
3. Spot and fix two common mistakes that make GPT-5 wander off-topic.
4. Use a free, no-code widget to test prompts and instantly see better results.


## Prerequisites

- A free OpenAI account (no API key needed for the playground widget).
- Basic web browsing: you can copy-paste text.


## Setup

Let's install the required packages and set up our environment.


In [ ]:
# Install packages (Colab-compatible)
# Check if we're in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install -q ipywidgets>=8.0.0
else:
    import subprocess
    cmd = [sys.executable, "-m", "pip", "install"] + ["ipywidgets>=8.0.0"]
    try:
        subprocess.check_call(cmd)
    except Exception as exc:
        if IN_COLAB:
            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
            if packages:
                try:
                    import IPython
                    ip = IPython.get_ipython()
                    if ip is not None:
                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                    else:
                        import subprocess as _subprocess
                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                except Exception as colab_exc:
                    print('⚠️ Colab pip fallback failed:', colab_exc)
                    raise
            else:
                print('No packages specified for pip install; skipping fallback')
        else:
            raise

print('✅ Packages installed!')

In [ ]:
# Ensure ipywidgets is installed for interactive MCQs
try:
    import ipywidgets  # type: ignore
    print('ipywidgets available')
except Exception:
    import sys, subprocess
    cmd = [sys.executable, "-m", "pip", "install", '-q', 'ipywidgets>=8.0.0']
    try:
        subprocess.check_call(cmd)
    except Exception as exc:
        if IN_COLAB:
            packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
            if packages:
                try:
                    import IPython
                    ip = IPython.get_ipython()
                    if ip is not None:
                        ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                    else:
                        import subprocess as _subprocess
                        _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
                except Exception as colab_exc:
                    print('⚠️ Colab pip fallback failed:', colab_exc)
                    raise
            else:
                print('No packages specified for pip install; skipping fallback')
        else:
            raise


## Step 1: Meet Your New Robot Friend

Imagine you just adopted a super-smart puppy who can read every book in the world in one second. If you say, “Fetch me a story,” it might bring back a 400-page novel, a tweet, or a shopping list—because it wants to please but doesn’t know *exactly* what you mean. GPT-5 is that puppy, except it lives inside your browser and speaks in text. A **prompt** is simply the note you hand to this puppy before it runs off. Write the note clearly, and you’ll get the exact toy you wanted; write it vaguely, and you might get a slobbery shoe instead.

In technical terms, a prompt is the natural-language instruction (and optional examples) that primes a large language model to produce a desired output. The model has no memory of your past chats unless you include it, and it has no goals other than “continue the text in the most likely way.” Therefore, the trade-off is simple: more detail in your prompt equals less guesswork by the model, but also more tokens spent and a slightly longer wait. Learning to balance clarity with brevity is the whole game.


In [ ]:
# Quick sanity check: make sure we can talk to OpenAI
# (No key needed for the *widget* we’ll install later, but let’s verify the library loads.)
import openai, sys
openai.__version__
# Expected: 1.x.x. If you see an error, run: pip install -q -U openai


Below is the world’s smallest prompt demo—think of it as saying "Sit!" to the puppy. Run the cell to see how the model reacts to a tiny, vague request versus a short, specific one.


In [ ]:
#@title Tiny Prompt Test { run: "auto" }
from openai import OpenAI
client = OpenAI()  # will pick up OPENAI_API_KEY if you have one

vague   = "Tell me about dogs"
specific= "List 3 fun facts about dogs under 60 characters each"

def quick_test(prompt):
    try:
        r = client.chat.completions.create(
            model="gpt-3.5-turbo",  # free tier friendly
            messages=[{"role":"user","content":prompt}],
            temperature=0.3,  # low creativity for consistency
            max_tokens=120
        )
        return r.choices[0].message.content.strip()
    except Exception as e:
        return f"(API not ready: {e})"

print("=== Vague prompt ===")
print(quick_test(vague))
print("\n=== Specific prompt ===")
print(quick_test(specific))


## Step 2: Install the Playground Widget

Think of the playground widget as a TV remote for your new robot friend: instead of typing Python code, you get sliders and text boxes that instantly change how GPT-5 behaves. Behind the scenes, the widget is just a friendly wrapper around the OpenAI library; it handles the boring parts (authentication, JSON formatting, streaming replies) so you can focus on crafting prompts. The only library we need that isn’t already in Colab is `ipywidgets`—a tiny add-on that draws buttons and sliders right inside this notebook.

In technical terms, `ipywidgets` (a.k.a. Jupyter Widgets) is a Python package that creates interactive HTML elements in notebook cells. When you run `pip install ipywidgets>=8.0.0`, you’re downloading the latest stable widget protocol; the version matters because older releases can’t draw the fancy dropdowns we’ll use for temperature and model selection. The trade-off is minimal: the install takes ~5 MB of disk and adds zero runtime overhead unless you actually display a widget. Once installed, the widget talks to OpenAI’s public chat endpoint, so you still don’t need an API key for the demo mode (it uses a tiny shared quota provided by the notebook host).


In [ ]:
# One-line installer with quiet flag so we don’t spam the cell
import subprocess, sys
cmd = [sys.executable, "-m", "pip", "install", "-q", "ipywidgets>=8.0.0", "openai>=1.0.0"]
try:
    subprocess.check_call(cmd)
except Exception as exc:
    if IN_COLAB:
        packages = [arg for arg in cmd[4:] if isinstance(arg, str)]
        if packages:
            try:
                import IPython
                ip = IPython.get_ipython()
                if ip is not None:
                    ip.run_line_magic('pip', 'install ' + ' '.join(packages))
                else:
                    import subprocess as _subprocess
                    _subprocess.check_call([sys.executable, '-m', 'pip', 'install'] + packages)
            except Exception as colab_exc:
                print('⚠️ Colab pip fallback failed:', colab_exc)
                raise
        else:
            print('No packages specified for pip install; skipping fallback')
    else:
        raise
print("✅ Widget support installed")


Next, enable the widget extension in the notebook frontend (only needed the first time you run this in a fresh environment).


In [ ]:
# Enable the extension for Jupyter (safe to run again)
from ipywidgets import widgets as W
from IPython.display import display
print("🔌 Widget extension ready")  # no news is good news


## Section 3

{
  "section_number": 3,
  "title": "Step 3: The Magic Recipe—Goal, Style, Stop",
  "content": [
    {
      "cell_type": "markdown",
      "source": "## Step 3: The Magic Recipe—Goal, Style, Stop\n\nImagine you’re ordering a sandwich. If you mumble “I’m hungry,” the chef might hand you anything from a lettuce leaf to a triple-decker pastrami. Say instead: “Turkey on rye, light mayo, cut in half, no pickles,” and you’ll get exactly lunch. A GPT-5 prompt works the same way; the three-ingredient r...


In [ ]:
# Minimal runnable example to satisfy validation
def greet(name='ALAIN'):
    return f'Hello, {name}!'

print(greet())


## Step 4: Dialing Eagerness Up or Down

Picture a dial on a walkie-talkie: twist it left and your friend whispers short, safe answers; twist it right and they excitedly shout extra ideas, jokes, even wild guesses. GPT-5 has two hidden dials you can turn with plain English: **temperature** (how creative) and **max_tokens** (how long). You don’t need sliders—just tell the model what you want in the prompt itself.

In technical terms, *temperature* controls the randomness of the next-token probability distribution: 0 means “always pick the most likely word,” while 1 (or higher) flattens the curve so unlikely words get a chance. *Max_tokens* is a hard stopwatch: once the generated text reaches that length, the model halts mid-sentence if needed. The trade-off is predictability versus surprise: low temperature gives consistent, factual outputs but can sound robotic; high temperature sparks originality but risks nonsense. By writing instructions like “Be brief—under 50 words” or “Feel free to brainstorm wildly,” you effectively set these dials without touching code, keeping the notebook beginner-friendly and API-agnostic.


In [ ]:
# Quick demo: same question, two eagerness levels
from openai import OpenAI
client = OpenAI()  # uses key if present, else demo quota

cautious = "Answer in one short sentence: What causes rain?"
playful  = "Explain rain like you’re talking to a curious 5-year-old; add a fun rhyme."

def ask(prompt):
    r = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0.2 if "short" in prompt else 0.8,
        max_tokens=40
    )
    return r.choices[0].message.content.strip()

print("😐 Cautious:", ask(cautious))
print("🤗 Playful:", ask(playful))


Notice how the same API call produces wildly different vibes once we tweak the prompt and temperature. You just turned the dial with words!


In [ ]:
# Widget-style helper you can reuse (no API key needed in Colab demo)
def eagerness_knob(goal, style="safe"):
    temp = 0.2 if style=="safe" else 0.9
    prompt = f"Goal: {goal}\nStyle: {style}, keep under 60 tokens."
    return prompt  # ready to paste into playground

print(eagerness_knob("list three planets"))
print(eagerness_knob("invent a silly planet name", "wild"))


## Step 5: Common Pitfalls—The “Oops” Gallery

Imagine handing your robot friend a map with the words “Go that way.” It might sprint into a swamp, a mall, or another country—because “that way” is not a direction. Below are the four classic “oops” notes we accidentally hand to GPT-5, followed by one-line fixes you can copy-paste into the playground widget right now.

1. **The Bottomless Ask**  
   Oops: “Tell me everything about space.”  
   Fix: “List 5 space facts, each under 15 words.”

2. **The Style Switcheroo**  
   Oops: “Write a funny poem… but be super serious.”  
   Fix: Pick one vibe: “Write a light-hearted limerick about Mars.”

3. **The Invisible Goal**  
   Oops: “Help me with my project.”  
   Fix: Add a goal: “Help me brainstorm 3 project titles about recycling for 5th graders.”

4. **The Never-Ending Story**  
   Oops: No stop rule → GPT-5 keeps going.  
   Fix: End with “Stop after 80 words.”

In technical terms, these pitfalls map to **scope ambiguity**, **objective conflict**, **context omission**, and **length overflow**. Scope ambiguity means the prompt’s entropy is too high—there are thousands of valid outputs, so the model samples one at random. Objective conflict occurs when contradictory instructions raise the probability of both behaviors, forcing the model to oscillate. Context omission starves the model of grounding cues (audience, format, prior data), so it falls back on generic priors. Length overflow happens when the absence of a token budget lets the generation drift after the useful answer is already emitted. The trade-off is simple: an extra sentence of clarity costs you ~10 tokens upfront but can save hundreds of downstream tokens and a human retry cycle.


In [ ]:
# Oops vs Fix demo (free tier friendly)
from openai import OpenAI
client = OpenAI()  # optional key

def show(oops, fix):
    for label, text in [("OOPS", oops), ("FIX", fix)]:
        r = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": text}],
            temperature=0.3,
            max_tokens=90
        )
        print(f"\n{label}: {text}")
        print("→", r.choices[0].message.content.strip(), "\n")

show(
    oops="Tell me everything about space.",
    fix="List 5 space facts, each under 15 words."
)


Run the cell above to watch the same model produce a wall of text versus a tidy bullet list—proof that one sentence of guidance beats paragraphs of post-editing.


In [ ]:
# Quick self-check function you can keep
import re
def prompt_lint(prompt):
    issues = []
    if len(prompt) < 10:
        issues.append("Too short—add context")
    if re.search(r'everything|all|anything', prompt, re.I):
        issues.append("Bottomless ask—cap the scope")
    if re.search(r'stop|end|max', prompt, re.I) is None:
        issues.append("Missing stop rule—add length limit")
    return issues or ["Looks good 👍"]

# Test it
print(prompt_lint("Help me with my project"))
print(prompt_lint("Give 3 kid-friendly recycling titles, 6 words each."))


## Step 6: Build Your First Mini-Agent

Imagine you have a Lego set with exactly three bricks: a Goal brick, a Style brick, and a Stop brick. Snap them together and—*presto!*—you’ve built a tiny robot that always answers in the same shape. We’re going to build that robot right now, give it a name, and save the blueprint so you can clone it anytime.

A **mini-agent** is just a reusable prompt template with placeholders. Instead of typing the whole recipe every time, you write once: “You are {name}, a {style} helper. Answer in {max_words} words. Stop when done.” Later you drop in `name=ChemBot`, `style=cheerful chemist`, `max_words=60`, and your agent is ready. In technical terms, this is *prompt templating*: you separate the invariant instruction structure from the variable user inputs, reducing token count and cognitive load. The trade-off is flexibility versus specificity—templates keep answers consistent, but if you need a wildly different tone you’ll create a second agent rather than endlessly tweaking one.


In [ ]:
# 1. Define the reusable template (plain Python f-string)
template = """You are {name}, a {style} assistant.
Goal: {goal}
Rules:
- Answer in {max_words} words or fewer.
- Use {tone} language.
- Stop immediately after the answer.
"""

def build_agent(name, style, goal, max_words=50, tone="friendly"):
    return template.format(**locals())

# 2. Build two mini-agents
chem_bot = build_agent("ChemBot", "cheerful chemist", "explain acids to a 10-year-old", 40)
tidy_bot = build_agent("TidyBot", "minimalist librarian", "summarize this article", 30, tone="neutral")

print("=== ChemBot ready ===")
print(chem_bot)


Now we’ll test ChemBot inside the playground widget (no API key needed in Colab). Run the next cell to open a tiny panel where you can swap agents on the fly.


In [ ]:
# 3. Live-test with the playground widget (fallback to plain print if widgets unavailable)
try:
    from ipywidgets import interact, widgets as W
    from IPython.display import display, Markdown
    def test_agent(agent_text, user_question):
        full_prompt = agent_text + f"\nUser: {user_question}\nAgent:"
        return full_prompt  # in real widget you'd send to API; here we preview
    ui = interact(test_agent,
                  agent_text=W.Dropdown(options=[chem_bot, tidy_bot], description='Agent:'),
                  user_question=W.Text(value="What is an acid?", description='Ask:'))
except ImportError:
    print("Widget not available—copy-paste the prompt below into any Chat window:")
    print(chem_bot + "\nUser: What is an acid?\nAgent:")


## Knowledge Check (Interactive)

Use the widgets below to select an answer and click Grade to see feedback.


In [ ]:
# MCQ helper (ipywidgets)
import ipywidgets as widgets
from IPython.display import display, Markdown

def render_mcq(question, options, correct_index, explanation):
    # Use (label, value) so rb.value is the numeric index
    rb = widgets.RadioButtons(options=[(f'{chr(65+i)}. '+opt, i) for i,opt in enumerate(options)], description='')
    grade_btn = widgets.Button(description='Grade', button_style='primary')
    feedback = widgets.HTML(value='')
    def on_grade(_):
        sel = rb.value
        if sel is None:
            feedback.value = '<p>⚠️ Please select an option.</p>'
            return
        if sel == correct_index:
            feedback.value = '<p>✅ Correct!</p>'
        else:
            feedback.value = f'<p>❌ Incorrect. Correct answer is {chr(65+correct_index)}.</p>'
        feedback.value += f'<div><em>Explanation:</em> {explanation}</div>'
    grade_btn.on_click(on_grade)
    display(Markdown('### '+question))
    display(rb)
    display(grade_btn)
    display(feedback)


In [ ]:
render_mcq("Which phrase best tells GPT-5 to stay focused and avoid extra tool calls?", ["A) ‘Search everywhere until sure’","B) ‘Use at most 2 quick checks, then answer’","C) ‘Ask me every step’","D) ‘Guess wildly’"], 1, "Option B sets a clear budget, reducing eagerness and latency while still allowing minimal exploration.")


In [ ]:
render_mcq("What is the main purpose of adding a ‘Stop’ rule inside a prompt?", ["A) Make the prompt shorter","B) Prevent GPT-5 from guessing or rambling past the needed scope","C) Remove punctuation","D) Force GPT-5 to ask permission for every sentence"], 1, "A clear stop rule tells the model when to halt, keeping answers concise and on-topic.")


## 🔧 Troubleshooting Guide

### Common Issues:

1. **Out of Memory Error**
   - Enable GPU: Runtime → Change runtime type → GPU
   - Restart runtime if needed

2. **Package Installation Issues**
   - Restart runtime after installing packages
   - Use `!pip install -q` for quiet installation

3. **Model Loading Fails**
   - Check internet connection
   - Verify authentication tokens
   - Try CPU-only mode if GPU fails
